# Maternal Health Risk Prediction - Baseline Models

This notebook trains and evaluates multiple baseline machine learning models for maternal health risk classification.

## Models Trained:
1. Logistic Regression
2. Decision Tree
3. Random Forest
4. Gradient Boosting
5. XGBoost
6. LightGBM

## Evaluation Metrics:
- Accuracy
- Precision
- Recall (especially for High Risk class)
- F1 Score
- AUC-ROC


In [ ]:
# Import libraries
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, classification_report, 
                             confusion_matrix, roc_curve)
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")


## 1. Load Processed Data


In [ ]:
# Load processed data
from data_processing import load_processed_data

X_train, X_val, X_test, y_train, y_val, y_test, scaler, feature_names = load_processed_data()

print(f"✓ Data loaded successfully!")
print(f"  Training set: {X_train.shape}")
print(f"  Validation set: {X_val.shape}")
print(f"  Test set: {X_test.shape}")
print(f"  Features: {feature_names}")


## 2. Define Models


In [ ]:
# Define baseline models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=42, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, 
                                           class_weight='balanced', n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, 
                                                    max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5,
                            random_state=42, eval_metric='mlogloss'),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5,
                              random_state=42, class_weight='balanced', verbose=-1)
}

print(f"✓ {len(models)} models defined")


## 3. Train and Evaluate Models


In [ ]:
# Training and evaluation
results = {}
trained_models = {}

for model_name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Training {model_name}...")
    print('='*60)
    
    # Train
    model.fit(X_train, y_train)
    
    # Predictions
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    y_test_proba = model.predict_proba(X_test)
    
    # Metrics
    val_accuracy = accuracy_score(y_val, y_val_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
    test_recall = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)
    test_f1 = f1_score(y_test, y_test_pred, average='weighted', zero_division=0)
    
    # High Risk recall (most important)
    high_risk_recall = recall_score(y_test, y_test_pred, labels=[2], average='macro', zero_division=0)
    
    # AUC
    try:
        test_auc = roc_auc_score(y_test, y_test_proba, multi_class='ovr', average='weighted')
    except:
        test_auc = 0.0
    
    # Store results
    results[model_name] = {
        'val_accuracy': val_accuracy,
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1,
        'test_auc': test_auc,
        'high_risk_recall': high_risk_recall
    }
    
    trained_models[model_name] = model
    
    # Print results
    print(f"\nValidation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Precision: {test_precision:.4f}")
    print(f"Test Recall: {test_recall:.4f}")
    print(f"Test F1 Score: {test_f1:.4f}")
    print(f"Test AUC: {test_auc:.4f}")
    print(f"High Risk Recall: {high_risk_recall:.4f}")

print("\n" + "="*60)
print("✓ All models trained successfully!")
print("="*60)


In [ ]:
# Create results dataframe
results_df = pd.DataFrame(results).T
results_df = results_df.round(4)

print("Model Performance Comparison:")
print("="*60)
print(results_df)

# Best model
best_model_name = results_df['test_f1'].idxmax()
best_f1_score = results_df['test_f1'].max()

print(f"\n✓ Best Model: {best_model_name} (F1 Score: {best_f1_score:.4f})")


In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

metrics = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1']
titles = ['Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1 Score']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[idx // 2, idx % 2]
    
    values = results_df[metric].sort_values(ascending=True)
    colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(values)))
    
    ax.barh(values.index, values.values, color=colors, edgecolor='black', alpha=0.8)
    ax.set_xlabel(title, fontsize=12, fontweight='bold')
    ax.set_title(f'{title} Comparison', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, v in enumerate(values.values):
        ax.text(v + 0.005, i, f'{v:.4f}', va='center')

plt.tight_layout()
plt.show()


## 5. Detailed Analysis of Best Model


In [ ]:
# Detailed evaluation of best model
best_model = trained_models[best_model_name]
y_test_pred_best = best_model.predict(X_test)

print(f"Detailed Analysis of {best_model_name}")
print("="*60)
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred_best, target_names=['Low Risk', 'Mid Risk', 'High Risk']))


In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred_best)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Low Risk', 'Mid Risk', 'High Risk'],
            yticklabels=['Low Risk', 'Mid Risk', 'High Risk'],
            cbar_kws={'label': 'Count'})
plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# Feature importance (if available)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nFeature Importance:")
    print(feature_importance)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance, x='Importance', y='Feature', palette='viridis')
    plt.title(f'Feature Importance - {best_model_name}', fontsize=14, fontweight='bold')
    plt.xlabel('Importance Score', fontsize=12)
    plt.tight_layout()
    plt.show()
else:
    print("\nFeature importance not available for this model type")


## 6. Confusion Matrices for All Models


In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, (model_name, model) in enumerate(trained_models.items()):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
               xticklabels=['Low', 'Mid', 'High'],
               yticklabels=['Low', 'Mid', 'High'],
               cbar_kws={'label': 'Count'})
    axes[idx].set_title(f'{model_name}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()


## Summary

Key findings from baseline model training:


In [ ]:
print("="*60)
print("BASELINE MODELS SUMMARY")
print("="*60)

print(f"\n✓ Best Model: {best_model_name}")
print(f"  F1 Score: {results[best_model_name]['test_f1']:.4f}")
print(f"  Accuracy: {results[best_model_name]['test_accuracy']:.4f}")
print(f"  High Risk Recall: {results[best_model_name]['high_risk_recall']:.4f}")

print("\n✓ Top 3 Models by F1 Score:")
top3 = results_df.nlargest(3, 'test_f1')[['test_f1', 'test_accuracy', 'high_risk_recall']]
print(top3)

print("\n✓ Next Steps:")
print("  1. Train deep learning model for comparison")
print("  2. Apply SHAP and LIME for explainability")
print("  3. Select final model for deployment")

print("\n" + "="*60)
